In [1]:
#pip install plotly

In [2]:
#pip install jupyter-dash

In [3]:
#pip install ipywidgets

In [4]:
# based on https://www.kaggle.com/code/dabawse/introduction-to-choropleth-maps-and-time-series
# and https://plotly.com/python/choropleth-maps/#using-geopandas-data-frames
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter

In [5]:
# needed to actually make stuff appear in jupyterlab
import plotly.io as pio
pio.renderers.default= 'iframe' #'iframe' or 'notebook' or 'colab' or 'jupyterlab'

In [6]:
#load geometry data of regions. Rename some of the regions to match the HPI data
# GeoJSON file is from https://github.com/martinjc/UK-GeoJSON/blob/master/json/administrative/sco/
regions_geometry = gpd.read_file("lad.json").set_index('LAD13NM')
regions_geometry.rename(index={'Aberdeen City': 'City of Aberdeen', 'Dundee City' :'City of Dundee', 'Glasgow City': 'City of Glasgow', 'Eilean Siar':'Na h-Eileanan Siar'},inplace=True)
regions_geometry

,LAD13CD,LAD13CDO,LAD13NMW,geometry
LAD13NM,,,,
Clackmannanshire,S12000005,00QF,None,"MULTIPOLYGON (((-3.81696 56.10242, -3.82170 56..."
Dumfries and Galloway,S12000006,00QH,None,"MULTIPOLYGON (((-4.41211 54.68063, -4.41274 54..."
East Ayrshire,S12000008,00QK,None,"POLYGON ((-4.49561 55.76300, -4.49404 55.76249..."
East Lothian,S12000010,00QM,None,"MULTIPOLYGON (((-2.78670 56.06567, -2.78476 56..."
East Renfrewshire,S12000011,00QN,None,"POLYGON ((-4.36747 55.81771, -4.36763 55.81660..."
Na h-Eileanan Siar,S12000013,00RJ,None,"MULTIPOLYGON (((-7.64974 56.78414, -7.64984 56..."
Falkirk,S12000014,00QQ,None,"POLYGON ((-3.79705 56.10033, -3.79929 56.09964..."
Fife,S12000015,00QR,None,"MULTIPOLYGON (((-3.30257 56.03095, -3.30251 56..."
Highland,S12000017,00QT,None,"MULTIPOLYGON (((-5.69535 56.50228, -5.69580 56..."


In [7]:
# Get house price data, extract Scotland regions, rename the column to match the geodata
df = pd.read_csv("Average-prices-2022-07.csv")
scotland = df[df['Area_Code'].str.match('S1')];
scotland = scotland.rename(columns={'Region_Name':'LAD13NM'})
scotland

,Date,LAD13NM,Area_Code,Average_Price,Monthly_Change,Annual_Change,Average_Price_SA
44031,2004-01-01,Moray,S12000020,69229.70100,NaN,NaN,NaN
44032,2004-01-01,Na h-Eileanan Siar,S12000013,39393.14602,NaN,NaN,NaN
44033,2004-01-01,Aberdeenshire,S12000034,81693.66964,NaN,NaN,NaN
44034,2004-01-01,Angus,S12000041,71248.99510,NaN,NaN,NaN
44035,2004-01-01,Argyll and Bute,S12000035,79248.64021,NaN,NaN,NaN
...,...,...,...,...,...,...,...
137388,2022-07-01,City of Glasgow,S12000049,175262.84556,2.053086,11.087222,NaN
137389,2022-07-01,North Lanarkshire,S12000050,142186.25461,3.306391,13.127429,NaN
137399,2022-07-01,Clackmannanshire,S12000005,168894.85265,-0.376147,11.877827,NaN
137400,2022-07-01,Dumfries and Galloway,S12000006,165012.25530,0.874348,14.532339,NaN


In [8]:
# not required, easier to keep geometry and data in separate dataframes
# scotland_with_geo = regions_geometry.merge(scotland, on ="LAD13NM")
# scotland_with_geo

In [9]:
# DON'T RUN THIS!!!! SEE BELOW! this ran for over 10 hours, took 6GB RAM but didn't complete.
#fig = px.choropleth(scotland_with_geo
#                    ,geojson=scotland_with_geo.geometry
#                    ,locations='LAD13NM'
#                    ,color='Average_Price'
#                    ,animation_frame='Date'
#                    ,center={'lat':57.0830876,'lon':-4.9235761}
#                    ,fitbounds='locations')
#fig.show()

In [10]:
small = scotland[scotland['Date'].str.startswith('2006-1') & scotland['LAD13NM'].str.startswith('F')]
small

,Date,LAD13NM,Area_Code,Average_Price,Monthly_Change,Annual_Change,Average_Price_SA
57794,2006-10-01,Falkirk,S12000014,97107.85461,-0.413904,12.274036,NaN
57795,2006-10-01,Fife,S12000047,112161.70960,0.561628,12.623781,NaN
58244,2006-11-01,Falkirk,S12000014,99392.18562,2.352365,14.852961,NaN
58245,2006-11-01,Fife,S12000047,113285.46110,1.001903,15.879783,NaN
58654,2006-12-01,Fife,S12000047,112225.29750,-0.935834,12.567707,NaN
58663,2006-12-01,Falkirk,S12000014,99234.42265,-0.158728,12.988601,NaN


In [11]:
# test to see if we're on the right lines...
fig = px.choropleth(small
                    ,geojson=regions_geometry
                    ,locations='LAD13NM'
                    ,color='Annual_Change'
                    ,animation_frame='Date'
                    ,center={'lat':57.0830876,'lon':-4.9235761}
                    ,fitbounds='locations')
fig.show()
#this looks better but even this took 10 seconds to run, and extra time to render (for 6 (2 regions*3 dates) regions)

In [12]:
#problem seems to be the complexity of the geometry. Try to simplify this by removing points close together.
# experiment to find something reasonable between good enough output and running time.
# tollerence 0.01 takes 2 seconds to render 6 geographies, 0.001 takes 4 seconds, no simplification takes 10s.
import copy

simple_regions_geometry = copy.deepcopy(regions_geometry)
simple_regions_geometry['geometry'] = simple_regions_geometry['geometry'].simplify(0.005)
simple_regions_geometry

,LAD13CD,LAD13CDO,LAD13NMW,geometry
LAD13NM,,,,
Clackmannanshire,S12000005,00QF,None,"MULTIPOLYGON (((-3.81696 56.10242, -3.81890 56..."
Dumfries and Galloway,S12000006,00QH,None,"MULTIPOLYGON (((-4.41211 54.68063, -4.41274 54..."
East Ayrshire,S12000008,00QK,None,"POLYGON ((-4.49561 55.76300, -4.48580 55.74968..."
East Lothian,S12000010,00QM,None,"MULTIPOLYGON (((-2.78670 56.06567, -2.75712 56..."
East Renfrewshire,S12000011,00QN,None,"POLYGON ((-4.36747 55.81771, -4.37607 55.81086..."
Na h-Eileanan Siar,S12000013,00RJ,None,"MULTIPOLYGON (((-7.64974 56.78414, -7.64984 56..."
Falkirk,S12000014,00QQ,None,"POLYGON ((-3.79705 56.10033, -3.76910 56.07665..."
Fife,S12000015,00QR,None,"MULTIPOLYGON (((-3.30257 56.03095, -3.29631 56..."
Highland,S12000017,00QT,None,"MULTIPOLYGON (((-5.69535 56.50228, -5.69679 56..."


In [13]:
fig = px.choropleth(small
                    ,geojson=simple_regions_geometry
                    ,locations='LAD13NM'
                    ,color='Average_Price'
                    ,animation_frame='Date'
                    ,center={'lat':57.0830876,'lon':-4.9235761}
                    ,fitbounds='locations'
                    ,basemap_visible=False #basemap is not as precise as the regions so it's of no benefit
                    ,range_color=[scotland['Average_Price'].min(),scotland['Average_Price'].max()]
                    ,width=800
                    ,height=800
                   )
fig.show()

In [14]:
# check projection being used. Use a small sample of dates so we don't wait for it to process
allregions_threedate = scotland[scotland['Date'].str.startswith('2006-1')]
fig = px.choropleth(allregions_threedate
                    ,geojson=simple_regions_geometry
                    ,locations='LAD13NM'
                    ,color='Average_Price'
                    ,animation_frame='Date'
                    ,center={'lat':57.0830876,'lon':-4.9235761}
                    ,fitbounds='locations'
                    ,basemap_visible=False #basemap is not as precise as the regions so it's of no benefit
                    ,range_color=[scotland['Average_Price'].min(),scotland['Average_Price'].max()]
                    ,width=1000
                    ,height=1000
                    ,projection='orthographic'
                   ) 
fig.show()
# looks decent enough! this took 4 seconds of kernel processing

In [19]:
# 111 seconds execution and a few more for rendering! note you may have to zoom out to see Shetland
statistic='Annual_Change'
fig = px.choropleth(scotland[scotland['Date'] > '2005-00-00'] # ignore first year as values are N/A
                    ,geojson=simple_regions_geometry
                    ,locations='LAD13NM'
                    ,color=statistic
                    ,animation_frame='Date'
                    ,center={'lat':57.0830876,'lon':-4.9235761}
                    ,fitbounds='locations'
                    ,basemap_visible=False #basemap is not as precise as the regions so it's of no benefit
                    ,range_color=[scotland[statistic].min(),scotland[statistic].max()]
                    ,width=1000
                    ,height=1000
                    ,projection='orthographic'
                   ) 
fig.show()

In [19]:
# 111 seconds execution and a few more for rendering! note you may have to zoom out to see Shetland
statistic='Annual_Change'
fig = px.choropleth(scotland[scotland['Date'] > '2005-00-00'] # ignore first year as values are N/A
                    ,geojson=simple_regions_geometry
                    ,locations='LAD13NM'
                    ,color=statistic
                    ,animation_frame='Date'
                    ,center={'lat':57.0830876,'lon':-4.9235761}
                    ,fitbounds='locations'
                    ,basemap_visible=False #basemap is not as precise as the regions so it's of no benefit
                    ,range_color=[scotland[statistic].min(),scotland[statistic].max()]
                    ,width=1000
                    ,height=1000
                    ,projection='orthographic'
                   ) 
fig.show()